# Lab 3

In [119]:
#Importing Dependencies
import pandas as pd
import numpy as np
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")

In [120]:
#initial look at the data
xmas

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,NaN,NaN,in a pear tree
1,2,second,dove,NaN,turtle,NaN
2,3,third,hen,NaN,french,NaN
3,4,fourth,bird,NaN,calling,NaN
4,5,fifth,ring,NaN,golden,NaN
5,6,sixth,goose,a-laying,NaN,NaN
6,7,seventh,swan,a-swimming,NaN,NaN
7,8,eighth,maid,a-milking,NaN,NaN
8,9,ninth,lady,dancing,NaN,NaN
9,10,tenth,lord,a-leaping,NaN,NaN


## Function 1: pluralize_gift()

In [121]:
def pluralize_gift(gift):
    """
    Returns the plural of a noun.
    
    Parameters
    ----------
    gift: str, A noun
    Checks for some irregular nouns
        
    Return
    ------
    str
        Plural version of noun
    """
    
    #Checking for irregular nouns that wouldn't be pluralized with just "s"
    if "oo" in gift:
        return gift.replace("oo", "ee")
    elif gift.endswith("y"):
        return gift[:-1] + "ies"
    #Adding s to regular nouns
    else:
        gift = gift + "s"
    return gift

In [122]:
#Testing function on irregular noun
print(pluralize_gift('lady'))
#Testing function on regular noun
print(pluralize_gift('piper'))

ladies
pipers


In [123]:
#vectorizing this function to test it on 'Gift.Item'
vec_plural_gift = np.vectorize(pluralize_gift)
print(vec_plural_gift(xmas['Gift.Item']))

['partridges' 'doves' 'hens' 'birds' 'rings' 'geese' 'swans' 'maids'
 'ladies' 'lords' 'pipers' 'drummers']


## Function 2: make_phrase()

In [124]:
#Creating a function that takes arguments and makes a coherent phrase out of them
def make_phrase(num, num_word, item, verb, adjective, location):
    """
    Returns a complete phrase based on specified parameters
    
    Parameters:
    num (int): Numeric number.
    num_word (str): The word representation of the number.
    item (str): An item, noun.
    verb (str): The verb describing the noun action.
    adjective (str): Adjective for noun.
    location (str): Location for the noun.
    
    Returns:
    str: The full phrase.
    """
    #Dictionary to be able to map the place numbers to strings of the number represented
    day_number_mapping = {
    'first': 'one',
    'second': 'two',
    'third': 'three',
    'fourth': 'four',
    'fifth': 'five',
    'sixth': 'six',
    'seventh': 'seven',
    'eighth': 'eight',
    'ninth': 'nine',
    'tenth': 'ten',
    'eleventh': 'eleven',
    'twelfth': 'twelve'
    }

    #Step 1: Replacing NAs with blank strings
    verb = verb if pd.notna(verb) else ""
    adjective = adjective if pd.notna(adjective) else ""
    location = location if pd.notna(location) else ""
    
    #Step 2: Making sure to only pluralize words that come after the first since this is for a specific song
    if num > 1:
        #implementing the previously made pluralize_gift function
        item = pluralize_gift(item)
    
    #Step 3: Assigning variable to get the numeric word representation of placeholding words
    num_word = day_number_mapping.get(num_word)
    
    # Step 4: Dealing with nuances in the first word to be coherent
    if num == 1:
        # Checking if the word starts with a vowel
        starts_with_vowel = item.lower()[0] in ['a', 'e', 'i', 'o', 'u']
        
        # Specifying word to start with "an" if it starts with a vowel, otherwise use "a"
        day_part = "an" if starts_with_vowel else "a"
        
    else:
        # If no conditions are met then it will just return num_word
        day_part = num_word
    
    # Step 5: Putting all pieces together and making a sentence
    parts = [day_part, adjective, item, verb, location]
    filtered_parts = [part for part in parts if part]  # Filter out empty parts
    phrase = ' '.join(filtered_parts)
    
    # Add a period at the end of the phrase when num == 1
    if num == 1:
        phrase += "."
    
    #making sure it actually returns the phrase at the end of the function
    return phrase

In [125]:
#testing the function on random things
make_phrase(num = 2, num_word = 'second', item = 'turtle dove', verb = "", adjective = "", location ="" )

'two turtle doves'

### Making Full_Phrase Column

In [126]:
#Utilizing lambda to be able to iterate through and make a new column in dataset with a full phrase
xmas['Full_Phrase'] = xmas.apply(lambda row: make_phrase(
                    num=row['Day'],
                    num_word=row['Day.in.Words'],
                    item=row['Gift.Item'],
                    verb=row['Verb'],
                    adjective=row['Adjective'],
                    location=row['Location']
                    ),axis=1)

#Checking what the full_phrase column looks like with its corresponding day
print(xmas[['Day', 'Full_Phrase']])

    Day                  Full_Phrase
0     1  a partridge in a pear tree.
1     2             two turtle doves
2     3            three french hens
3     4           four calling birds
4     5            five golden rings
5     6           six geese a-laying
6     7       seven swans a-swimming
7     8        eight maids a-milking
8     9          nine ladies dancing
9    10          ten lords a-leaping
10   11         eleven pipers piping
11   12     twelve drummers drumming


## Function 3: sing_day()

In [127]:
#creating a function that will iteratively sing a song when you specify a day, with a particular dataset
def sing_day(dataset, num, phrase_col):
    """
    Returns versions of the song "12 Days of Christmas" for the specified day with a specific dataset

    Parameters
    ----------
    dataset, DataFrame
        The dataset containing the phrases for each day
    num, int
        The day number
    phrase_col : str
        The column name in the dataset that contains the phrases for each day

    Return
    ------
    str
        The song verse for the specified day
    """

    #making a dictionary to be able to call the placeholder of any number.
    num_to_word = {1:"first", 2:"second", 3:"third", 4:"fourth", 5:"fifth", 6:"sixth",
                   7:"seventh", 8:"eighth", 9:"ninth", 10:"tenth", 11:"eleventh", 12:"twelfth"}

    # Step 1: Setting up the introduction line of the song to be sang converting "1" to "first" etc.
    num_word = num_to_word[num]
    intro = "On the " + num_word + " day of Christmas, my true love sent to me:\n"

    # Step 2: Sing the gift phrases up to the given day making sure to reverse the order of the gifts
    gifts_list = dataset[phrase_col].iloc[:num].tolist()[::-1]

    #Adding and before the first gift if there is more than one gift to be sung about
    if num > 1:
        gifts_list[-1] = "and " + gifts_list[-1]

    #Adding a new line after each phrase
    gifts = ",\n".join(gifts_list)

    #Concatenating everything to be the full song
    song = f"{intro}{gifts}"
    
    #returing the full song to enjoy
    return song

In [128]:
#tesiting out the function
print(sing_day(xmas, 2, 'Full_Phrase'))

On the second day of Christmas, my true love sent to me:
two turtle doves,
and a partridge in a pear tree.


In [129]:
#Singing full song :)
print('\n\n'.join(map(lambda day: sing_day(xmas, day, 'Full_Phrase'), range(1,13))))

On the first day of Christmas, my true love sent to me:
a partridge in a pear tree.

On the second day of Christmas, my true love sent to me:
two turtle doves,
and a partridge in a pear tree.

On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fourth day of Christmas, my true love sent to me:
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fifth day of Christmas, my true love sent to me:
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the sixth day of Christmas, my true love sent to me:
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the seventh day of Christmas, my true love sent to me:
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a 

## Using Function on another DataSet

In [130]:
#time to import the other data set to test our functions
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")

In [131]:
#intitial look at other data set
xmas2.head()

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,email,NaN,NaN,from Cal Poly
1,2,second,point,NaN,meal,NaN
2,3,third,pen,NaN,lost,NaN
3,4,fourth,review,NaN,course,NaN
4,5,fifth,exam,NaN,practice,NaN


In [132]:
#Creating thhe full phrase column on the other dataset also
xmas2['Full_Phrase'] = xmas2.apply(
    lambda row: make_phrase(
        num=row['Day'],
        num_word=row['Day.in.Words'],
        item=row['Gift.Item'],
        verb=row['Verb'],
        adjective=row['Adjective'],
        location=row['Location']
    ),axis=1)

#Checking if the full_phrase column was added correctly
print(xmas2[['Day', 'Full_Phrase']])

    Day                 Full_Phrase
0     1     an email from Cal Poly.
1     2             two meal points
2     3             three lost pens
3     4         four course reviews
4     5         five practice exams
5     6         six graders grading
6     7     seven seniors stressing
7     8        eight moms a-calling
8     9        nine parties bumping
9    10        ten loads of laundry
10   11  eleven friends goodbye-ing
11   12       twelve hours sleeping


In [133]:
#testing the sing_day function on this dataset to see what phrases there are
print(sing_day(xmas2, 9, 'Full_Phrase'))

On the ninth day of Christmas, my true love sent to me:
nine parties bumping,
eight moms a-calling,
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.


In [134]:
#Singing full song :)
print('\n\n'.join(map(lambda day: sing_day(xmas2, day, 'Full_Phrase'), range(1, 13))))

On the first day of Christmas, my true love sent to me:
an email from Cal Poly.

On the second day of Christmas, my true love sent to me:
two meal points,
and an email from Cal Poly.

On the third day of Christmas, my true love sent to me:
three lost pens,
two meal points,
and an email from Cal Poly.

On the fourth day of Christmas, my true love sent to me:
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the fifth day of Christmas, my true love sent to me:
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the sixth day of Christmas, my true love sent to me:
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the seventh day of Christmas, my true love sent to me:
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On t

In [135]:
#last checkpoint :)
print(sing_day(xmas, num = 3, phrase_col = "Full_Phrase"))

On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.
